<a href="https://colab.research.google.com/github/AABarodkin/python_mftids22/blob/main/%22Environment_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 5.3 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 


In [46]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Агент#1 который всегда камень

In [47]:
%%writefile rock_agent.py


#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py



Агент#2 который всегда ножницы

In [48]:
%%writefile scissors_agent.py


#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


Агент#3 котрый всегда бумага

In [49]:
%%writefile paper_agent.py


#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


Агент#4, который производит то же самое действие, что и оппонент на прошлом ходу

In [50]:
%%writefile copy_opponent.py

import random
def copy_opponent(observation, configuration):

    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Агент#5 который делает победное действие прошлого хода оппонента

In [51]:
%%writefile wincopy_opponent.py

import random
def wincopy_opponent(observation, configuration):

    if observation.step > 0:
      if observation.lastOpponentAction==0:
        return 1
      if observation.lastOpponentAction==1:
        return 2
      if observation.lastOpponentAction==2:
        return 0
    else:
        return random.randrange(0, configuration.signs)

Overwriting wincopy_opponent.py


Агент #6 который делает проигрошное действие прошлого хода оппонента

In [52]:
%%writefile losscopy_opponent.py
#0 - rock
#1 - paper
#2 - scissors
import random
def losscopy_opponent(observation, configuration):

    if observation.step > 0:
      if observation.lastOpponentAction==0:
        return 2
      if observation.lastOpponentAction==1:
        return 0
      if observation.lastOpponentAction==2:
        return 1
    else:
        return random.randrange(0, configuration.signs)

Overwriting losscopy_opponent.py


Агент #7 который статистично оценивает какое у оппонента наиболее частое действие за весь период игры и ставит всегда выйгрышное действие против него

In [53]:
%%writefile statist_opponent.py
#0 - rock
#1 - paper
#2 - scissors
import random
stat={0:0,1:0,2:0}
def statist(observation, configuration):
  global stat
  if observation.step > 0:
    stat[observation.lastOpponentAction]=stat.get(observation.lastOpponentAction,0)+1
    k=[stat[0],stat[1],stat[2]]
    if max(k)==0:
      return 1
    if max(k)==1:
      return 2
    if max(k)==2:
      return 0
  else:
    return random.randrange(0, configuration.signs)

Overwriting statist_opponent.py


Агент #8 который статистично оценивает какое у оппонента наиболее редкое действие за весь период игры и ставит всегда выйгрышное действие против него

In [54]:
%%writefile minstatist_opponent.py
#0 - rock
#1 - paper
#2 - scissors
import random
stat={0:0,1:0,2:0}
def minstatist(observation, configuration):
  global stat
  if observation.step > 0:
    stat[observation.lastOpponentAction]=stat.get(observation.lastOpponentAction,0)+1
    k=[stat[0],stat[1],stat[2]]
    if min(k)==0:
      return 1
    if min(k)==1:
      return 2
    if min(k)==2:
      return 0
  else:
    return random.randrange(0, configuration.signs)

Writing minstatist_opponent.py


Создадим список всех агентов

In [55]:
agents=["rock_agent.py", "scissors_agent.py", "paper_agent.py", "copy_opponent.py", "wincopy_opponent.py", "losscopy_opponent.py", 'statist_opponent.py', 'minstatist_opponent.py' ]

Создадим пустой список для очков

In [56]:
scores=[0 for x in range(8)]


In [57]:
for i in range(len(agents)):
  for j in range(i+1, len(agents)):
    result=evaluate("rps", [agents[i], agents[j]], configuration={"episodeSteps": 100})

    scores[agents.index(agents[i])]+=result[0][0]
    scores[agents.index(agents[j])]+=result[0][1]

    


Выводим результаты

In [58]:
print('Winner: ', agents[scores.index(max(scores))], ' with' , max(scores))
print('summary table:')
for i in range(len(agents)):
  print(agents[i], ':', scores[i])

Winner:  wincopy_opponent.py  with 636.0
summary table:
rock_agent.py : -99.0
scissors_agent.py : -2.0
paper_agent.py : 96.0
copy_opponent.py : 0.0
wincopy_opponent.py : 636.0
losscopy_opponent.py : -631.0
statist_opponent.py : -93.0
minstatist_opponent.py : 93.0
